# Data Transformation - Item Sales Aggregation

This notebook contains the data preprocessing pipeline for the cafe supply forecasting project. It includes:

1. Sales Data Merger - Merges sales data from different language sources
2. Sales Data Cleaner - Cleans and standardizes sales data
3. Item Sales Aggregator - Aggregates daily sales quantities for each item

In [1]:
# Import common libraries
import pandas as pd
import numpy as np
import os
from typing import Dict, Optional, List, Set, Tuple
from datetime import datetime

## 1. Sales Data Merger

This section merges two sales CSV files with different languages (Indonesian and English) into a single standardized dataset.

In [2]:
def translate_indonesian_to_english(df_indonesian: pd.DataFrame) -> pd.DataFrame:
    """
    Translate Indonesian column names and values to English

    Args:
        df_indonesian: DataFrame with Indonesian column names and values

    Returns:
        DataFrame with translated English columns and values
    """
    # Column mapping from Indonesian to English
    column_mapping = {
        'Tanggal': 'Date',
        'Nomor struk': 'Receipt number',
        'Jenis struk': 'Receipt type',
        'Kategori': 'Category',
        'SKU': 'SKU',
        'Barang': 'Item',
        'Varian': 'Variant',
        'Pemodifikasi diterapkan': 'Modifiers applied',
        'Kuantitas': 'Quantity',
        'Penjualan Kotor': 'Gross sales',
        'Diskon': 'Discounts',
        'Penjualan bersih': 'Net sales',
        'Harga pokok': 'Cost of goods',
        'Laba kotor': 'Gross profit',
        'Pajak': 'Taxes',
        'Jenis pesanan': 'Dining option',
        'POS': 'POS',
        'Toko': 'Store',
        'Nama Kasir': 'Cashier name',
        'Nama Pelanggan': 'Customer name',
        'Kontak Pelanggan': 'Customer contacts',
        'Komentar': 'Comment',
        'Keadaan': 'Status'
    }
    
    # Rename columns
    df_indonesian = df_indonesian.rename(columns=column_mapping)
    
    # Translate specific values
    value_mappings = {
        'Receipt type': {
            'Penjualan': 'Sale'
        },
        'Dining option': {
            'Makan di tempat': 'Dine in'
        },
        'Status': {
            'Ditutup': 'Closed'
        }
    }
    
    # Apply value translations
    for column, mapping in value_mappings.items():
        if column in df_indonesian.columns:
            df_indonesian[column] = df_indonesian[column].replace(mapping)
    
    return df_indonesian

def parse_date(date_str) -> pd.Timestamp:
    """
    Parse different date formats to standard datetime

    Args:
        date_str: Date string in various formats

    Returns:
        Parsed datetime object or NaT if parsing fails
    """
    if pd.isna(date_str):
        return pd.NaT

    date_str = str(date_str).strip()

    # Try different date formats
    formats_to_try = [
        '%d/%m/%y %H.%M',  # Indonesian format: 13/05/25 23.43
        '%d/%m/%y %H:%M',  # Alternative Indonesian format
        '%m/%d/%y %I:%M %p',  # English format: 9/25/25 10:07 PM
        '%m/%d/%Y %I:%M %p',  # English format with 4-digit year
    ]

    for fmt in formats_to_try:
        try:
            return pd.to_datetime(date_str, format=fmt)
        except:
            continue

    # If none of the specific formats work, try pandas auto-detection
    try:
        return pd.to_datetime(date_str)
    except:
        return pd.NaT

def clean_numeric_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean and convert numeric columns to proper numeric types

    Args:
        df: DataFrame with numeric columns that may contain non-numeric characters

    Returns:
        DataFrame with cleaned numeric columns
    """
    numeric_columns = [
        'Quantity', 'Gross sales', 'Discounts', 'Net sales',
        'Cost of goods', 'Gross profit', 'Taxes'
    ]

    for col in numeric_columns:
        if col in df.columns:
            # Remove any non-numeric characters except decimal point
            df[col] = df[col].astype(str).str.replace(r'[^\d.]', '', regex=True)
            # Convert to numeric, errors='coerce' will turn invalid values to NaN
            df[col] = pd.to_numeric(df[col], errors='coerce')

    return df

def merge_sales_files(file1_path: str, file2_path: str, output_path: str) -> Optional[pd.DataFrame]:
    """
    Merge two sales CSV files with different languages

    Args:
        file1_path: Path to Indonesian sales CSV file
        file2_path: Path to English sales CSV file
        output_path: Path where merged CSV should be saved

    Returns:
        Merged DataFrame or None if merge fails
    """
    print(f'Reading file 1: {file1_path}')
    print(f'Reading file 2: {file2_path}')

    # Read both files
    try:
        # Indonesian file (first file)
        df1 = pd.read_csv(file1_path, sep=';')
        print(f'File 1 loaded successfully. Shape: {df1.shape}')

        # English file (second file)
        df2 = pd.read_csv(file2_path, sep=',')
        print(f'File 2 loaded successfully. Shape: {df2.shape}')

    except Exception as e:
        print(f'Error reading files: {e}')
        return None

    # Translate Indonesian file to English
    print('Translating Indonesian file to English...')
    df1_translated = translate_indonesian_to_english(df1)

    # Clean numeric columns in both dataframes
    print('Cleaning numeric columns...')
    df1_translated = clean_numeric_columns(df1_translated)
    df2 = clean_numeric_columns(df2)

    # Parse dates in both dataframes
    print('Parsing dates...')
    df1_translated['Date'] = df1_translated['Date'].apply(parse_date)
    df2['Date'] = df2['Date'].apply(parse_date)
    
    # Ensure both dataframes have the same columns
    all_columns = set(df1_translated.columns) | set(df2.columns)
    
    for col in all_columns:
        if col not in df1_translated.columns:
            df1_translated[col] = np.nan
        if col not in df2.columns:
            df2[col] = np.nan
    
    # Reorder columns to match English file order
    column_order = [
        'Date', 'Receipt number', 'Receipt type', 'Category', 'SKU', 'Item', 
        'Variant', 'Modifiers applied', 'Quantity', 'Gross sales', 'Discounts', 
        'Net sales', 'Cost of goods', 'Gross profit', 'Taxes', 'Dining option', 
        'POS', 'Store', 'Cashier name', 'Customer name', 'Customer contacts', 
        'Comment', 'Status'
    ]
    
    df1_translated = df1_translated[column_order]
    df2 = df2[column_order]
    
    # Combine the dataframes
    print('Combining dataframes...')
    combined_df = pd.concat([df1_translated, df2], ignore_index=True)

    # Sort by date
    print('Sorting by date...')
    combined_df = combined_df.sort_values('Date', na_position='last')

    # Reset index
    combined_df = combined_df.reset_index(drop=True)

    # Save the merged file
    print(f'Saving merged file to: {output_path}')
    combined_df.to_csv(output_path, index=False)

    print(f'Merge completed successfully!')
    print(f'Total records: {len(combined_df)}')
    print(f'Date range: {combined_df["Date"].min()} to {combined_df["Date"].max()}')
    
    return combined_df

In [3]:
# Execute the sales data merger
base_path = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
file1_path = os.path.join(base_path, 'data', 'raw', 'sales', 'receipts-by-item-2022-01-01-2025-06-30.csv')
file2_path = os.path.join(base_path, 'data', 'raw', 'sales', 'receipts-by-item-2025-05-01-2025-09-25.csv')
output_path = os.path.join(base_path, 'data', 'processed', 'sales_data.csv')

# Create output directory if it doesn't exist
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Merge the files
merged_data = merge_sales_files(file1_path, file2_path, output_path)

if merged_data is not None:
    print('\nFirst few rows of merged data:')
    print(merged_data.head())

    print('\nData summary:')
    print(merged_data.info())

Reading file 1: /Users/wyk/Documents/personal/thesis/cafe-supply-forecasting/data/raw/sales/receipts-by-item-2022-01-01-2025-06-30.csv
Reading file 2: /Users/wyk/Documents/personal/thesis/cafe-supply-forecasting/data/raw/sales/receipts-by-item-2025-05-01-2025-09-25.csv
File 1 loaded successfully. Shape: (48588, 23)
File 2 loaded successfully. Shape: (10189, 23)
Translating Indonesian file to English...
Cleaning numeric columns...


/var/folders/dw/hqyrj5817x1bnh5y864bj0gh0000gp/T/ipykernel_7566/3424459581.py:138: DtypeWarning: Columns (19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(file1_path, sep=';')


Parsing dates...
Combining dataframes...
Sorting by date...
Saving merged file to: /Users/wyk/Documents/personal/thesis/cafe-supply-forecasting/data/processed/sales_data.csv
Merge completed successfully!
Total records: 58777
Date range: 2022-01-01 14:31:00 to 2025-09-25 22:07:00

First few rows of merged data:
                 Date Receipt number Receipt type Category    SKU  \
0 2022-01-01 14:31:00         Jan-82         Sale      Tea     41   
1 2022-01-01 15:34:00         Jan-83         Sale      NaN  10044   
2 2022-01-01 15:34:00         Jan-83         Sale      NaN  10049   
3 2022-01-01 16:16:00         Jan-84         Sale    snack     84   
4 2022-01-01 16:22:00         Jan-85         Sale    Other   3300   

                Item  Variant  Modifiers applied  Quantity  Gross sales  ...  \
0      Hot Tea Ajeng      NaN                NaN       1.0      15000.0  ...   
1    Rose pink latte      NaN                NaN       1.0      23000.0  ...   
2  Roti bakar klasik      NaN    

## 2. Sales Data Cleaner

This section cleans and standardizes sales data to match the menu BOM.

In [4]:
class SalesDataCleaner:
    """Clean and standardize sales data based on menu BOM"""

    def __init__(self, sales_path: str, menu_bom_path: str):
        """
        Initialize the cleaner with file paths

        Args:
            sales_path: Path to sales data CSV
            menu_bom_path: Path to menu BOM CSV
        """
        self.sales_path = sales_path
        self.menu_bom_path = menu_bom_path
        self.sales_df = pd.read_csv(sales_path)
        self.menu_bom_df = pd.read_csv(menu_bom_path)

        # Build mappings
        self.rename_map = self._build_rename_mapping()
        self.package_items = self._build_package_mapping()
        self.active_items = self._get_active_items()

        # Track statistics
        self.stats = {
            'original_records': len(self.sales_df),
            'renamed_records': 0,
            'expanded_packages': 0,
            'expanded_items': 0,
            'discontinued_items': 0,
            'removed_records': 0
        }

    def _build_rename_mapping(self) -> Dict[str, str]:
        """
        Build mapping of old names to standardized names

        Returns:
            Dictionary mapping old item names to new standardized names
        """
        # Define rename mappings (sales name → BOM name)
        rename_map = {
            # Tea items
            'lemon tea hot': 'Lemon Hot',
            'lemon tea ice': 'Lemon Ice',
            'hot tea ajeng': 'Ajeng Hot',
            'ice tea ajeng': 'Ajeng Ice',
            'hot tea anindya': 'Anindya Hot',
            'ice tea anindya': 'Anindya Ice',

            # Coffee items
            'kopi susu panas': 'Kopi Susu Hot',
            'kopi susu ice': 'Kopi Susu Ice',
            'ice cappucino': 'Cappucino Ice',
            'latte': 'Latte Hot',
            'ice latte': 'Latte Ice',
            'picolo': 'Piccolo',
            'long black hot': 'Black Hot',
            'long black ice': 'Black Ice',

            # Milk-based items
            'red velvet': 'Red Velvet Ice',
            'susu coklat': 'Coklat Hot',

            # V60 variants (all consolidate to v60)
            'v60 hacienda natural': 'v60',
            'v60 argopuro': 'v60',
            'v60 finca': 'v60',

            # Noodle items
            'mie goreng telur': 'Mie Goreng',
            'mie rebus telur': 'Mie Rebus',

            # Rice items
            'nasi goreng djawa': 'Nasi Goreng Jawa',
            'nasi ayam daun jeruk': 'Ayam Daun Jeruk',
            'nasi ayam rempah': 'Ayam Rempah',

            # Main course items
            'ayam mentega (paket)': 'Ayam Mentega',
            'ayam curry (paket)': 'Ayam Curry',
            'ayam dauh jeruk (paket)': 'Ayam Daun Jeruk',

            # Snacks
            'cireng isi': 'Cireng',

            # Desserts
            'pannacotta': 'Panacotta',

            # Additional items
            'add vanilla ice cream': 'Add Ice Cream Vanilla',
            'add telur ceplok': 'Add Telur',
            'add telur dadar': 'Add Telur',
            'add caramel topping': 'Add Topping Caramel',

            # Case variations for consistency
            'kentang goreng': 'Kentang Goreng',
            'lychee tea hot': 'Lychee Tea Hot',
            'lychee tea ice': 'Lychee Tea Ice',
            'matcha ice': 'Matcha Ice',
            'mie goreng': 'Mie Goreng',
            'mie rebus': 'Mie Rebus',
            'nasi gila': 'Nasi Gila',
            'red velvet ice': 'Red Velvet Ice',
            'roti bakar coklat': 'Roti Bakar Coklat',
            'roti bakar klasik': 'Roti Bakar Klasik',
            'roti bakar strawberry': 'Roti Bakar Strawberry',
            'singkong keju': 'Singkong Keju',
            'tahu tuna': 'Tahu Tuna',
            'taro ice': 'Taro Ice',
            'tempe mendoan': 'Tempe Mendoan',
            'pisang goreng aren': 'Pisang Goreng Aren',
            'vietnam drip': 'Vietnam Drip',
            'waffle vanilla': 'Waffle Vanilla',
            'add keju': 'Add Keju',
            'add kerupuk': 'Add Kerupuk',
            'coklat ice': 'Coklat Ice',
            'cheese cake': 'Cheese Cake',
        }

        return rename_map

    def _build_package_mapping(self) -> Dict[str, List[Tuple[str, float]]]:
        """
        Build mapping for package deals that need to be split

        Returns:
            Dictionary mapping package names to list of (item, quantity multiplier) tuples
        """
        package_map = {
            # Mie with telur - add telur as separate item
            'mie goreng telur': [
                ('Mie Goreng', 1.0),
                ('Add Telur', 1.0)
            ],
            'mie rebus telur': [
                ('Mie Rebus', 1.0),
                ('Add Telur', 1.0)
            ],
            # Package deals
            'paket ayam teriyaki + lemon tea': [
                ('Ayam Teriyaki', 1.0),
                ('Lemon Ice', 1.0)
            ],
            'paket ayam curry + lemon tea': [
                ('Ayam Curry', 1.0),
                ('Lemon Ice', 1.0)
            ],
        }

        return package_map

    def _get_active_items(self) -> Set[str]:
        """
        Get set of active items from menu BOM

        Returns:
            Set of normalized active item names
        """
        active_items = set()

        # Get unique items from menu BOM
        for item_name in self.menu_bom_df['Item'].unique():
            # Normalize by stripping whitespace and converting to lowercase
            normalized = item_name.strip().lower()
            active_items.add(normalized)

        return active_items

    def _normalize_item_name(self, item_name: str) -> str:
        """
        Normalize item name for comparison

        Args:
            item_name: Original item name

        Returns:
            Normalized item name (lowercase, stripped)
        """
        if pd.isna(item_name):
            return ''
        return str(item_name).strip().lower()

    def standardize_names(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Standardize item names in sales data

        Args:
            df: DataFrame to standardize

        Returns:
            DataFrame with standardized item names
        """
        print('\n' + '-' * 80)
        print('STEP 1: STANDARDIZING ITEM NAMES')
        print('-' * 80)

        # Create a copy to work with
        standardized_df = df.copy()

        # Apply simple renames first
        renamed_count = 0
        for old_name, new_name in self.rename_map.items():
            # Case-insensitive matching
            mask = standardized_df['Item'].str.lower() == old_name.lower()
            if mask.any():
                count = mask.sum()
                standardized_df.loc[mask, 'Item'] = new_name
                renamed_count += count
                print(f'  ✓ Renamed: "{old_name}" → "{new_name}" ({count} records)')

        self.stats['renamed_records'] = renamed_count

        # Handle package items that need to be split
        expanded_rows = []
        rows_to_remove = []

        for idx, row in standardized_df.iterrows():
            item_lower = str(row['Item']).lower()

            if item_lower in self.package_items:
                # Mark this row for removal
                rows_to_remove.append(idx)

                # Create new rows for each component
                components = self.package_items[item_lower]
                original_qty = row['Quantity']

                for component_item, qty_multiplier in components:
                    new_row = row.copy()
                    new_row['Item'] = component_item
                    new_row['Quantity'] = original_qty * qty_multiplier
                    expanded_rows.append(new_row)

                print(f'  ✓ Expanded: "{row["Item"]}" → {len(components)} items ({original_qty} qty each)')

        # Remove package items
        if rows_to_remove:
            standardized_df = standardized_df.drop(rows_to_remove)
            self.stats['expanded_packages'] = len(rows_to_remove)

        # Add expanded rows
        if expanded_rows:
            expanded_df = pd.DataFrame(expanded_rows)
            standardized_df = pd.concat([standardized_df, expanded_df], ignore_index=True)
            self.stats['expanded_items'] = len(expanded_rows)

        # Sort by date
        standardized_df = standardized_df.sort_values('Date').reset_index(drop=True)

        print(f'\nStandardization Summary:')
        print(f'  Simple renames: {renamed_count} records')
        print(f'  Package expansions: {len(rows_to_remove)} packages → {len(expanded_rows)} items')
        print(f'  Total records: {len(df)} → {len(standardized_df)}')

        return standardized_df

    def identify_discontinued_items(self, df: pd.DataFrame) -> Tuple[Set[str], pd.DataFrame, pd.DataFrame]:
        """
        Identify items in sales data that are not in menu BOM

        Args:
            df: DataFrame to check

        Returns:
            Tuple of (set of discontinued item names, DataFrame with discontinued items stats, DataFrame with valid items)
        """
        print('\n' + '-' * 80)
        print('STEP 2: IDENTIFYING DISCONTINUED ITEMS')
        print('-' * 80)

        # Get all unique items from sales data
        sales_items = df['Item'].dropna().unique()

        # Find discontinued items (in sales but not in menu BOM)
        discontinued_items = set()
        valid_items = set()

        for item in sales_items:
            normalized = self._normalize_item_name(item)
            if normalized and normalized not in self.active_items:
                discontinued_items.add(item)  # Store original name for display
            else:
                valid_items.add(item)

        self.stats['discontinued_items'] = len(discontinued_items)

        # Create statistics DataFrame for discontinued items
        discontinued_stats = []
        for item in discontinued_items:
            item_data = df[df['Item'] == item]
            stats = {
                'Item': item,
                'Total_Quantity_Sold': item_data['Quantity'].sum(),
                'Total_Transactions': len(item_data),
                'First_Sale_Date': item_data['Date'].min(),
                'Last_Sale_Date': item_data['Date'].max()
            }
            discontinued_stats.append(stats)

        discontinued_df = pd.DataFrame(discontinued_stats)

        # Sort by total quantity sold (descending)
        if not discontinued_df.empty:
            discontinued_df = discontinued_df.sort_values('Total_Quantity_Sold', ascending=False)

        print(f'\n✓ Valid items (found in BOM): {len(valid_items)}')
        print(f'✗ Discontinued items (NOT in BOM): {len(discontinued_items)}')

        return discontinued_items, discontinued_df, df

    def remove_discontinued_items(self, df: pd.DataFrame, discontinued_items: Set[str]) -> pd.DataFrame:
        """
        Remove discontinued items from sales data

        Args:
            df: DataFrame to clean
            discontinued_items: Set of discontinued item names to remove

        Returns:
            Cleaned DataFrame with discontinued items removed
        """
        # Filter out discontinued items
        cleaned_df = df[~df['Item'].isin(discontinued_items)].copy()

        self.stats['removed_records'] = len(df) - len(cleaned_df)

        return cleaned_df

    def save_cleaned_data(self, cleaned_df: pd.DataFrame, output_path: str):
        """
        Save cleaned sales data to CSV

        Args:
            cleaned_df: Cleaned DataFrame
            output_path: Path where cleaned CSV should be saved
        """
        # Create output directory if needed
        os.makedirs(os.path.dirname(output_path), exist_ok=True)

        # Save to CSV
        cleaned_df.to_csv(output_path, index=False)

        print(f'\n✓ Cleaned data saved to: {output_path}')
        print(f'  Total records: {len(cleaned_df)}')

In [5]:
# Execute the sales data cleaner
base_path = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
sales_path = os.path.join(base_path, 'data', 'processed', 'sales_data.csv')
menu_bom_path = os.path.join(base_path, 'data', 'raw', 'bom', 'menu_bom.csv')
output_path = os.path.join(base_path, 'data', 'processed', 'sales_data_cleaned.csv')

print('=' * 80)
print('SALES DATA CLEANER')
print('=' * 80)
print(f'\nSales data: {sales_path}')
print(f'Menu BOM: {menu_bom_path}')
print(f'Output: {output_path}')

# Initialize cleaner
cleaner = SalesDataCleaner(sales_path, menu_bom_path)

print(f'\nOriginal records: {len(cleaner.sales_df):,}')

# Step 1: Standardize names
standardized_df = cleaner.standardize_names(cleaner.sales_df)

# Step 2: Identify discontinued items
discontinued_items, discontinued_df, current_df = cleaner.identify_discontinued_items(standardized_df)

# Print discontinued items report
if discontinued_items:
    print(f'\nTotal discontinued items: {len(discontinued_items)}')
    print(f'Total transactions affected: {discontinued_df["Total_Transactions"].sum():.0f}')
    print(f'Total quantity sold: {discontinued_df["Total_Quantity_Sold"].sum():.0f}')
    
    # Display top discontinued items
    print('\nTop 10 discontinued items:')
    display(discontinued_df.head(10))
else:
    print('\n✓ No discontinued items found!')

# Step 3: Handle discontinued items (removing them)
print('\n' + '-' * 80)
print('STEP 3: HANDLING DISCONTINUED ITEMS')
print('-' * 80)

if discontinued_items:
    print('\nRemoving discontinued items...')
    final_df = cleaner.remove_discontinued_items(current_df, discontinued_items)
    print(f'✓ Removed {cleaner.stats["removed_records"]:,} records')
else:
    print('\n✓ No discontinued items to remove')
    final_df = current_df

# Save cleaned data
print('\n' + '-' * 80)
print('STEP 4: SAVING CLEANED DATA')
print('-' * 80)
cleaner.save_cleaned_data(final_df, output_path)

# Print final summary
print('\n' + '=' * 80)
print('FINAL SUMMARY')
print('=' * 80)

print(f'\nOriginal records: {cleaner.stats["original_records"]:,}')
print(f'\nStandardization:')
print(f'  - Renamed records: {cleaner.stats["renamed_records"]:,}')
print(f'  - Expanded packages: {cleaner.stats["expanded_packages"]:,} → {cleaner.stats["expanded_items"]:,} items')

print(f'\nDiscontinued items:')
print(f'  - Items found: {cleaner.stats["discontinued_items"]:,}')
print(f'  - Records removed: {cleaner.stats["removed_records"]:,}')

final_records = cleaner.stats["original_records"] + cleaner.stats["expanded_items"] - cleaner.stats["removed_records"]
print(f'\nFinal records: {final_records:,}')
print('=' * 80)

print('\n✓ Sales data cleaning complete!')

SALES DATA CLEANER

Sales data: /Users/wyk/Documents/personal/thesis/cafe-supply-forecasting/data/processed/sales_data.csv
Menu BOM: /Users/wyk/Documents/personal/thesis/cafe-supply-forecasting/data/raw/bom/menu_bom.csv
Output: /Users/wyk/Documents/personal/thesis/cafe-supply-forecasting/data/processed/sales_data_cleaned.csv

Original records: 58,777

--------------------------------------------------------------------------------
STEP 1: STANDARDIZING ITEM NAMES
--------------------------------------------------------------------------------
  ✓ Renamed: "lemon tea hot" → "Lemon Hot" (1004 records)
  ✓ Renamed: "lemon tea ice" → "Lemon Ice" (984 records)
  ✓ Renamed: "hot tea ajeng" → "Ajeng Hot" (499 records)
  ✓ Renamed: "ice tea ajeng" → "Ajeng Ice" (226 records)
  ✓ Renamed: "hot tea anindya" → "Anindya Hot" (207 records)
  ✓ Renamed: "ice tea anindya" → "Anindya Ice" (112 records)
  ✓ Renamed: "kopi susu panas" → "Kopi Susu Hot" (715 records)
  ✓ Renamed: "kopi susu ice" → "Kopi 

,Item,Total_Quantity_Sold,Total_Transactions,First_Sale_Date,Last_Sale_Date
110,Americano Ice,1003.0,981,2022-01-05 17:05:00,2025-05-13 18:08:00
129,Air mineral,655.0,440,2022-01-25 14:54:00,2025-05-12 22:27:00
111,Americano Hot,399.0,376,2022-01-10 18:49:00,2025-05-13 23:43:00
112,Ayam suwir,365.0,342,2022-10-19 16:41:00,2025-05-12 20:01:00
114,Rose Tea Hot,283.0,277,2022-08-17 11:54:00,2025-04-17 23:12:00
71,Air Mineral,261.0,235,2023-01-06 18:20:00,2025-09-24 14:43:00
76,Nasi,175.0,148,2022-01-03 20:48:00,2025-05-12 17:45:00
69,Air Mineral besar,165.0,142,2024-12-14 17:29:00,2025-05-12 19:44:00
109,PANNACOTTA 50 cup,160.0,3,2023-06-28 16:15:00,2023-07-14 14:38:00
59,Rose Tea Ice,132.0,129,2024-05-01 14:08:00,2025-04-15 21:49:00



--------------------------------------------------------------------------------
STEP 3: HANDLING DISCONTINUED ITEMS
--------------------------------------------------------------------------------

Removing discontinued items...
✓ Removed 4,604 records

--------------------------------------------------------------------------------
STEP 4: SAVING CLEANED DATA
--------------------------------------------------------------------------------

✓ Cleaned data saved to: /Users/wyk/Documents/personal/thesis/cafe-supply-forecasting/data/processed/sales_data_cleaned.csv
  Total records: 54209

FINAL SUMMARY

Original records: 58,777

Standardization:
  - Renamed records: 29,559
  - Expanded packages: 36 → 72 items

Discontinued items:
  - Items found: 138
  - Records removed: 4,604

Final records: 54,245

✓ Sales data cleaning complete!


## 3. Item Sales Aggregator

This section aggregates daily sales quantities for each item instead of calculating raw material requirements.

In [6]:
class ItemSalesAggregator:
    """Aggregate daily sales quantities for each item"""

    def __init__(self, sales_path: str):
        """
        Initialize the aggregator with sales data path

        Args:
            sales_path: Path to cleaned sales data CSV
        """
        self.sales_df = pd.read_csv(sales_path)

    def aggregate_daily_sales(self) -> pd.DataFrame:
        """
        Aggregate sales data to get daily quantities for each item

        Returns:
            DataFrame with columns: Date, Item, Quantity_Sold
        """
        print('Starting item sales aggregation...')

        # Make a copy to avoid modifying original
        df = self.sales_df.copy()
        
        # Convert Date column to datetime and extract just the date
        df['Date'] = pd.to_datetime(df['Date']).dt.date
        
        # Clean Item names
        df['Item'] = df['Item'].str.strip()
        
        # Consolidate espresso variants - treat 'espresso bon-bon' as 'espresso'
        df['Item'] = df['Item'].str.replace(r'^espresso bon-bon$', 'espresso', case=False, regex=True)
        
        # Filter out items with 'add' prefix (case insensitive) - these are modifiers
        initial_count = len(df)
        df = df[~df['Item'].str.lower().str.startswith('add')]
        filtered_count = initial_count - len(df)
        if filtered_count > 0:
            print(f"Filtered out {filtered_count} transactions with 'add' prefix (modifiers)")
        
        # Filter out discontinued items - cheese cake
        initial_count = len(df)
        df = df[~df['Item'].str.lower().str.contains('cheese cake')]
        filtered_count = initial_count - len(df)
        if filtered_count > 0:
            print(f"Filtered out {filtered_count} transactions for discontinued item 'cheese cake'")

        # Group by Date and Item, sum the Quantity
        aggregated_df = df.groupby(['Date', 'Item'])['Quantity'].sum().reset_index()

        # Rename columns for clarity
        aggregated_df = aggregated_df.rename(columns={'Quantity': 'Quantity_Sold'})

        # Sort by Date and Item
        aggregated_df = aggregated_df.sort_values(['Date', 'Item']).reset_index(drop=True)

        print(f'Aggregation complete!')
        print(f'Total unique dates: {aggregated_df["Date"].nunique()}')
        print(f'Total unique items: {aggregated_df["Item"].nunique()}')
        print(f'Total rows in output: {len(aggregated_df)}')

        return aggregated_df

    def save_aggregated_data(self, output_path: str):
        """
        Aggregate sales data and save results to CSV

        Args:
            output_path: Path where the output CSV should be saved
        """
        result_df = self.aggregate_daily_sales()

        # Create output directory if it doesn't exist
        os.makedirs(os.path.dirname(output_path), exist_ok=True)

        # Save to CSV
        result_df.to_csv(output_path, index=False)
        print(f'\nResults saved to: {output_path}')

In [7]:
# Execute the item sales aggregator
base_path = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
sales_path = os.path.join(base_path, 'data', 'processed', 'sales_data_cleaned.csv')
output_path = os.path.join(base_path, 'data', 'processed', 'daily_item_sales.csv')

# Initialize aggregator
aggregator = ItemSalesAggregator(sales_path)

# Aggregate and save
aggregator.save_aggregated_data(output_path)

Starting item sales aggregation...
Filtered out 1051 transactions with 'add' prefix (modifiers)
Filtered out 101 transactions for discontinued item 'cheese cake'
Aggregation complete!
Total unique dates: 1357
Total unique items: 78
Total rows in output: 31563

Results saved to: /Users/wyk/Documents/personal/thesis/cafe-supply-forecasting/data/processed/daily_item_sales.csv


## Summary

This notebook has successfully executed the complete data preprocessing pipeline:

1. **Sales Data Merger**: Combined Indonesian and English sales data into a single standardized dataset
2. **Sales Data Cleaner**: Standardized item names, expanded package deals, and removed discontinued items
3. **Item Sales Aggregator**: Aggregated daily sales quantities for each item

The processed data is now ready for further analysis and forecasting model development.